<a href="https://colab.research.google.com/github/dc-koreauniv-9/NLP/blob/master/190730%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%ED%95%9C%EA%B5%AD%EC%96%B4-%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!git clone https://github.com/e9t/nsmc.git

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [0]:
# github에 있는 데이터 가지고 오기
# id : 고유 id
# document : 내용
# label : 0 - 부정, 1 - 긍정 (결과가 조금 다를 수 있다.)

!cat ./nsmc/ratings_train.txt | head -n 10

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1


In [0]:

def read_data(filename):
  with open(filename,'r') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    # txt 파일의 헤더(id document label) 제외하기
    data = data[1:]
  return data



In [0]:
train_data = read_data("./nsmc/ratings_train.txt")
test_data = read_data("./nsmc/ratings_test.txt")

# 간단하게 3개만 출력
print(train_data[0:3])

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'], ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'], ['10265843', '너무재밓었다그래서보는것을추천한다', '0']]


In [0]:
print(len(train_data))
print(len(train_data[0]))

150000
3


# 데이터 전처리
- KoNLPy 라이브러리를 이용 형태소 분석

In [0]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 2.7MB/s 
     |████████████████████████████████| 2.7MB 26.2MB/s 


In [0]:
import konlpy
import json
import os
from pprint import pprint
from konlpy.tag import Okt
okt = Okt()

/usr/local/lib/python3.6/dist-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [0]:
# 영화 평 데이터에 형태소 분석을 통해서 품사를 태깅해주는 작업
# 품사를 태깅 : '더빙/Noun' -> None 같이 형태소의 뜻과 문맥을 고려하여 그것을 마크업
# 데이터의 양이 크기 때문에 작업을 반복하지 않기 위해서 json파일로 저장

train_data = train_data[:10000]
test_data = test_data[:1000]

def tokenize(doc):
   return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

if os.path.isfile('train_docs.json'):
  with open('train_docs.json') as f:
    train_docs = json.load(f)
  with open('test_docs.json') as f:
    test_docw = json.load(f)
else:
    train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
    test_docs = [(tokenize(row[1]), row[2]) for row in test_data]
    
    with open('train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

pprint(train_docs[0])

(['아/Exclamation',
  '더빙/Noun',
  '../Punctuation',
  '진짜/Noun',
  '짜증나다/Adjective',
  '목소리/Noun'],
 '0')


In [0]:
# 분석한 데이터의 토큰의 갯수를 확인하기
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

146795


In [0]:
# nltk 라이브러리를 통해서 전처리를 하기
# most_common을 사용해서 가장 많이 사용되는 단어를 추출한다.
import nltk
text = nltk.Text(tokens, name = 'NMSC')

print(len(text.tokens)) # 전체 토큰의 개수
print(len(set(text.tokens))) #중복을 제외한 토큰의 개수
pprint(text.vocab().most_common(10))

146795
13496
[('./Punctuation', 4791),
 ('영화/Noun', 3368),
 ('하다/Verb', 2829),
 ('이/Josa', 2624),
 ('보다/Verb', 2576),
 ('의/Josa', 2123),
 ('../Punctuation', 1949),
 ('가/Josa', 1789),
 ('에/Josa', 1771),
 ('을/Josa', 1587)]


#One Hot Encoding
- 데이터에는 수치형 데이터와 텍스트 데이터나 범주형데이터가있다.
- 머신러닝이나 딥러닝에 데이터가 사용되기 위해서는 수치화가 필요하다.
- 범주형 데이터 같은 경우 수치화가 필요한데 조건에 성립하는 하나의 데이터만 1 나머지는 0으로 채워주는것을 의미한다.

# CountVectorization
- token count를 matrix로 변환하는 클래스이다.
- 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 인코딩한 벡터를 만든다. -> 단어수를 세어  Bag을 만든다.
- 여기서는 one hot encoding이 아닌  countvectorization을 사용한다.

In [0]:
import numpy as np

# 시간 절약을 위하여 10000개의  most_common(10000) 사용한다.
selected_words = [f[0] for f in text.vocab().most_common(10000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

In [0]:
# 데이터를 float형으로 변환시켜주어야 전처리 과정이 끝난다.

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [56]:
# 64개의 유닛을 가지고 긍정의 리뷰일 확률을 출력한다.
# 10회 학습시킨다.
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)


W0729 15:07:39.355564 140012916492160 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 15:07:39.480202 140012916492160 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
10000/10000 [==============================] - 1s 112us/sample - loss: 0.6060 - binary_accuracy: 0.7379
Epoch 2/10
10000/10000 [==============================] - 0s 33us/sample - loss: 0.3999 - binary_accuracy: 0.8587
Epoch 3/10
10000/10000 [==============================] - 0s 34us/sample - loss: 0.2946 - binary_accuracy: 0.8965
Epoch 4/10
10000/10000 [==============================] - 0s 34us/sample - loss: 0.2319 - binary_accuracy: 0.9187
Epoch 5/10
10000/10000 [==============================] - 0s 34us/sample - loss: 0.1868 - binary_accuracy: 0.9358
Epoch 6/10
10000/10000 [==============================] - 0s 35us/sample - loss: 0.1530 - binary_accuracy: 0.9460
Epoch 7/10
10000/10000 [==============================] - 0s 35us/sample - loss: 0.1272 - binary_accuracy: 0.9580
Epoch 8/10
10000/10000 [==============================] - 0s 33us/sample - loss: 0.1069 - binary_accuracy: 0.9650
Epoch 9/10
10000/10000 [==============================] - 0s 34us/sample - loss: 0.0896

In [57]:
# 테스트 데이터로 확인해본 결과 79%의 성능을 보여준다. 
results

[0.7803858957886696, 0.794]

#예측 함수 만들기
- 새로운 데이터를 가지고와서 결과를 예측한다. 

In [0]:
#확률이 0.5보다 크면 긍정, 그보다 작으면 부정이라고 예측한다.
def predict_pos(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print("[{}]는 {:.2f}% 확률로 긍정 리뷰이지 않을까 추측해본다\n".format(review, score * 100))
    else:
        print("[{}]는 {:.2f}% 확률로 부정 리뷰이지 않을까 추측해본다\n".format(review, (1 - score) * 100))

In [67]:
predict_pos("뭐 이런 영화가 다 있죠? 정말 최악이군요")

[뭐 이런 영화가 다 있죠? 정말 최악이군요]는 99.94% 확률로 부정 리뷰이지 않을까 추측해본다



In [68]:
predict_pos("제 인생영화 입니다 추천합니다.!!")

[제 인생영화 입니다 추천합니다.!!]는 83.36% 확률로 긍정 리뷰이지 않을까 추측해본다

